# HRN Quick Test

**목적**: HRN 모델 동작 검증 + Google Drive 캐시로 런타임 재시작 시 빠른 복구

- 최초 실행: 설치 10~30분 (환경에 따라 다름)
- **런타임 재시작 후: ~2분** (Drive 캐시 활용)

수정된 문제들:
- PyTorch 2.6+ `torch.load` weights_only 기본값 변경
- `oss2` 모듈 누락
- `moviepy` 2.0 API 변경
- `nvdiffrast` EGL 초기화 실패 (GL→CUDA)
- `pytorch3d` CUDA 12.x pre-built wheel 부재

In [ ]:
# ============================================================
# Cell 0: 환경 설정 + Drive 마운트
# ============================================================
import os, sys, pathlib

# ★ PyTorch 2.6+ torch.load weights_only 문제 해결
os.environ['TORCH_FORCE_NO_WEIGHTS_ONLY_LOAD'] = '1'

# ★ CUDA 라이브러리 경로 설정 (libcudart.so 로딩 오류 방지)
_cuda_paths = []
for _p in ['/usr/local/cuda/lib64',
           '/usr/local/cuda/targets/x86_64-linux/lib',
           '/usr/local/cuda/lib']:
    if os.path.isdir(_p):
        _cuda_paths.append(_p)
for _p in sorted(pathlib.Path('/usr/local/lib').glob('python*/dist-packages/nvidia/*/lib')):
    _cuda_paths.append(str(_p))

_existing = os.environ.get('LD_LIBRARY_PATH', '')
_new = ':'.join(_cuda_paths)
os.environ['LD_LIBRARY_PATH'] = f'{_new}:{_existing}' if _existing else _new

# CUDA_HOME 설정
for _ch in ['/usr/local/cuda', '/usr/local/cuda-12.8', '/usr/local/cuda-12']:
    if os.path.isdir(_ch):
        os.environ['CUDA_HOME'] = _ch
        break

# libcudart 미리 로드 (런타임 링커 오류 방지)
import ctypes
for _p in _cuda_paths:
    for _f in sorted(pathlib.Path(_p).glob('libcudart.so.*'), reverse=True):
        try:
            ctypes.CDLL(str(_f), mode=ctypes.RTLD_GLOBAL)
            print(f'libcudart pre-loaded: {_f}')
            break
        except OSError:
            continue
    else:
        continue
    break

import torch
print(f'Python: {sys.version}')
print(f'PyTorch: {torch.__version__}')
print(f'CUDA: {torch.version.cuda}')
print(f'GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else "NO GPU!"}')

# Google Drive 마운트
from google.colab import drive
drive.mount('/content/drive')

DRIVE_ROOT = '/content/drive/MyDrive'
CACHE_ROOT = os.path.join(DRIVE_ROOT, 'BISTOOL_pip_cache')
os.makedirs(CACHE_ROOT, exist_ok=True)

# 캐시 키: Python + CUDA 버전 조합
CACHE_KEY = f'py3{sys.version_info.minor}_cu{torch.version.cuda.replace(".", "")}'
print(f'\n캐시 키: {CACHE_KEY}')
print(f'캐시 경로: {CACHE_ROOT}')

In [ ]:
# ============================================================
# Cell 1: 의존성 설치 (Drive 캐시 활용)
# ★ 최초: 10~30분 / 재시작: ~2분
# ============================================================
import time, glob, shutil
_start = time.time()

# ── 1) 누락 의존성 (빠름, 캐시 불필요) ──
print('[1/5] 누락 의존성...')
!pip install -q oss2 moviepy==1.0.3

# ── 2) 시스템 패키지 (apt .deb 캐시) ──
import subprocess
_uver = subprocess.run(['lsb_release', '-rs'], capture_output=True, text=True).stdout.strip().replace('.', '')
DEB_CACHE = os.path.join(CACHE_ROOT, 'apt_debs', f'ubuntu{_uver}')
DEB_MARKER = os.path.join(DEB_CACHE, '.complete')
APT_PKGS = 'build-essential ninja-build freeglut3-dev libgl1-mesa-dev libgles2-mesa-dev libglew-dev libegl1-mesa-dev'

if os.path.exists(DEB_MARKER):
    print('[2/5] 시스템 패키지 (캐시)...')
    !dpkg -i {DEB_CACHE}/*.deb > /dev/null 2>&1 || true
    !apt-get -f install -y -qq > /dev/null 2>&1
else:
    print('[2/5] 시스템 패키지 (설치 + 캐시 저장)...')
    os.makedirs(DEB_CACHE, exist_ok=True)
    !apt-get update -qq > /dev/null 2>&1
    !apt-get install -y -qq --download-only {APT_PKGS} > /dev/null 2>&1
    !cp /var/cache/apt/archives/*.deb {DEB_CACHE}/ 2>/dev/null || true
    !apt-get install -y -qq {APT_PKGS} > /dev/null 2>&1
    with open(DEB_MARKER, 'w') as f:
        f.write('ok')

# ── 3) nvdiffrast (wheel 캐시 + 로딩 검증) ──
NV_CACHE = os.path.join(CACHE_ROOT, 'nvdiffrast_wheels', CACHE_KEY)
os.makedirs(NV_CACHE, exist_ok=True)

# CUDA_HOME 설정 (빌드 시 필수)
for _ch in ['/usr/local/cuda', '/usr/local/cuda-12.8', '/usr/local/cuda-12']:
    if os.path.isdir(_ch):
        os.environ['CUDA_HOME'] = _ch
        break

def _verify_nvdiffrast():
    """nvdiffrast가 실제로 CUDA 컨텍스트에서 동작하는지 검증"""
    try:
        import importlib
        if 'nvdiffrast' in sys.modules:
            importlib.reload(sys.modules.get('nvdiffrast.torch', __import__('nvdiffrast.torch')))
        else:
            import nvdiffrast.torch
        return True
    except (ImportError, OSError) as e:
        print(f'  ⚠️ nvdiffrast 로딩 실패: {e}')
        return False

_nv_ok = False
try:
    import nvdiffrast.torch
    _nv_ok = True
    print('[3/5] nvdiffrast 이미 설치됨')
except (ImportError, OSError):
    pass

if not _nv_ok:
    # 캐시에서 설치 시도
    cached_nv = sorted(glob.glob(os.path.join(NV_CACHE, 'nvdiffrast*.whl')))
    if cached_nv:
        print('[3/5] nvdiffrast (캐시)...')
        os.system(f'pip install -q "{cached_nv[-1]}"')
        _nv_ok = _verify_nvdiffrast()
        if not _nv_ok:
            print('  ⚠️ 캐시 wheel 호환 안 됨 → 삭제 후 재빌드')
            for f in cached_nv:
                os.remove(f)

    if not _nv_ok:
        print('[3/5] nvdiffrast (빌드 + 캐시 저장)...')
        !pip install -q ninja
        # 기존 설치 제거
        !pip uninstall -y nvdiffrast 2>/dev/null || true
        TMP = '/tmp/nv_whl'
        os.makedirs(TMP, exist_ok=True)
        # 최신 버전 빌드 (v0.3.5 핀 제거)
        !pip wheel --no-build-isolation --no-deps \
            "git+https://github.com/NVlabs/nvdiffrast.git" \
            -w {TMP} 2>&1
        wheels = glob.glob(f'{TMP}/nvdiffrast*.whl')
        if wheels:
            os.system(f'pip install -q "{wheels[0]}"')
            if _verify_nvdiffrast():
                shutil.copy2(wheels[0], os.path.join(NV_CACHE, os.path.basename(wheels[0])))
                _nv_ok = True
        if not _nv_ok:
            # fallback: 직접 클론 + 설치
            !rm -rf /tmp/nvdiffrast_src
            !git clone --quiet https://github.com/NVlabs/nvdiffrast.git /tmp/nvdiffrast_src
            !pip install --no-build-isolation -q /tmp/nvdiffrast_src
            _nv_ok = _verify_nvdiffrast()

if _nv_ok:
    print('  ✅ nvdiffrast 검증 완료')
else:
    print('  ❌ nvdiffrast 설치/검증 실패!')

# ── 4) PyTorch3D (wheel 캐시) ──
PT3D_CACHE = os.path.join(CACHE_ROOT, 'pytorch3d_wheels', CACHE_KEY)
os.makedirs(PT3D_CACHE, exist_ok=True)

try:
    import pytorch3d
    print(f'[4/5] PyTorch3D {pytorch3d.__version__} 이미 설치됨')
except ImportError:
    !pip install -q fvcore iopath
    cached_pt3d = sorted(glob.glob(os.path.join(PT3D_CACHE, 'pytorch3d*.whl')))
    if cached_pt3d:
        print('[4/5] PyTorch3D (캐시)...')
        os.system(f'pip install -q "{cached_pt3d[-1]}"')
    else:
        print('[4/5] PyTorch3D (설치 + 캐시 저장)...')
        # 서드파티 wheel 시도
        ret = os.system(
            'pip install -q --extra-index-url '
            'https://miropsota.github.io/torch_packages_builder '
            'pytorch3d 2>/dev/null'
        )
        if ret != 0:
            # 소스 빌드
            print('  서드파티 wheel 없음, 소스 빌드 중... (15~20분)')
            TMP = '/tmp/pt3d_whl'
            os.makedirs(TMP, exist_ok=True)
            !pip wheel --no-build-isolation --no-deps \
                "git+https://github.com/facebookresearch/pytorch3d.git" \
                -w {TMP}
            wheels = glob.glob(f'{TMP}/pytorch3d*.whl')
            if wheels:
                os.system(f'pip install -q "{wheels[0]}"')
                shutil.copy2(wheels[0], os.path.join(PT3D_CACHE, os.path.basename(wheels[0])))
                print(f'  💾 Drive 캐시 저장 완료')
        else:
            # 서드파티 wheel 성공 → 캐시 저장
            import pytorch3d
            whl_path = pytorch3d.__file__
            # pip cache에서 wheel 찾기
            cached_pip = glob.glob('/root/.cache/pip/wheels/**/pytorch3d*.whl', recursive=True)
            if cached_pip:
                shutil.copy2(cached_pip[0], os.path.join(PT3D_CACHE, os.path.basename(cached_pip[0])))

# ── 5) ModelScope ──
try:
    from modelscope.pipelines import pipeline
    print('[5/5] ModelScope 이미 설치됨')
except ImportError:
    print('[5/5] ModelScope 설치...')
    !pip install -q "modelscope[cv]" -f https://modelscope.oss-cn-beijing.aliyuncs.com/releases/repo.html

_elapsed = time.time() - _start
print(f'\n✅ 전체 설치 완료! ({_elapsed:.0f}초)')

In [ ]:
# ============================================================
# Cell 2: 설치 검증 + GL→CUDA 패치
# ============================================================
import site, glob

# 설치 검증
errors = []
for name, imp in [('nvdiffrast', 'nvdiffrast.torch'), ('pytorch3d', 'pytorch3d'),
                   ('modelscope', 'modelscope.pipelines'), ('moviepy', 'moviepy.editor')]:
    try:
        __import__(imp)
        print(f'✅ {name}')
    except ImportError as e:
        errors.append(name)
        print(f'❌ {name}: {e}')

if errors:
    print(f'\n❌ {len(errors)}개 실패 - Cell 1 재실행 필요')
else:
    print('\n✅ 의존성 확인 완료!')

# GL → CUDA context 패치 (Colab에서 EGL 초기화 실패 방지)
_patched = 0
for sp in site.getsitepackages():
    for py_file in glob.glob(os.path.join(sp, 'modelscope', '**', '*.py'), recursive=True):
        try:
            with open(py_file, 'r', encoding='utf-8', errors='ignore') as f:
                content = f.read()
            if 'RasterizeGLContext' in content:
                with open(py_file, 'w', encoding='utf-8') as f:
                    f.write(content.replace('RasterizeGLContext', 'RasterizeCudaContext'))
                _patched += 1
        except Exception:
            pass
print(f'GL→CUDA 패치: {_patched}개 파일')

In [ ]:
# ============================================================
# Cell 3: HRN 모델 초기화 (Drive 캐시)
# ★ 최초: ~1GB 다운로드 / 재시작: Drive에서 즉시 로딩
# ============================================================
import os, sys, pathlib, cv2, time
os.environ['TORCH_FORCE_NO_WEIGHTS_ONLY_LOAD'] = '1'

# ★ CUDA 라이브러리 경로 재확인 (런타임에서 이 셀만 재실행할 경우 대비)
_cuda_paths = []
for _p in ['/usr/local/cuda/lib64',
           '/usr/local/cuda/targets/x86_64-linux/lib',
           '/usr/local/cuda/lib']:
    if os.path.isdir(_p):
        _cuda_paths.append(_p)
for _p in sorted(pathlib.Path('/usr/local/lib').glob('python*/dist-packages/nvidia/*/lib')):
    _cuda_paths.append(str(_p))
_existing = os.environ.get('LD_LIBRARY_PATH', '')
_new = ':'.join(_cuda_paths)
if _new not in _existing:
    os.environ['LD_LIBRARY_PATH'] = f'{_new}:{_existing}' if _existing else _new

import ctypes
for _p in _cuda_paths:
    for _f in sorted(pathlib.Path(_p).glob('libcudart.so.*'), reverse=True):
        try:
            ctypes.CDLL(str(_f), mode=ctypes.RTLD_GLOBAL)
            break
        except OSError:
            continue
    else:
        continue
    break

from moviepy.editor import ImageSequenceClip
from modelscope.models.cv.face_reconstruction.utils import write_obj
from modelscope.outputs import OutputKeys
from modelscope.pipelines import pipeline
from modelscope.utils.constant import Tasks

# 모델 캐시를 Google Drive에 저장
MODEL_CACHE = os.path.join(DRIVE_ROOT, 'BISTOOL_Models', 'modelscope_cache')
os.makedirs(MODEL_CACHE, exist_ok=True)
os.environ['MODELSCOPE_CACHE'] = MODEL_CACHE

HRN_MODEL_ID = 'damo/cv_resnet50_face-reconstruction'
cached = os.path.join(MODEL_CACHE, 'hub', HRN_MODEL_ID.replace('/', os.sep))
if os.path.exists(cached):
    print(f'✅ HRN 모델 Drive 캐시 발견: {cached}')
else:
    print('⬇️ HRN 모델 최초 다운로드 중 (~1GB, Drive에 캐시됨)...')

_t = time.time()
face_reconstruction = pipeline(
    Tasks.face_reconstruction,
    model=HRN_MODEL_ID,
    model_revision='v2.0.0-HRN'
)
print(f'✅ HRN 모델 로딩 완료! ({time.time()-_t:.1f}초)')

In [ ]:
# ============================================================
# Cell 4: 테스트 추론 (공식 예제 이미지)
# ============================================================
def save_results(result, save_root):
    os.makedirs(save_root, exist_ok=True)
    mesh = result[OutputKeys.OUTPUT]['mesh']
    texture_map = result[OutputKeys.OUTPUT_IMG]
    mesh['texture_map'] = texture_map
    write_obj(os.path.join(save_root, 'hrn_mesh_mid.obj'), mesh)
    frame_list = result[OutputKeys.OUTPUT]['frame_list']
    video = ImageSequenceClip(sequence=frame_list, fps=30)
    video.write_videofile(
        os.path.join(save_root, 'rotate.mp4'), fps=30, audio=False)
    del frame_list
    vis_image = result[OutputKeys.OUTPUT]['vis_image']
    cv2.imwrite(os.path.join(save_root, 'vis_image.jpg'), vis_image)
    print(f'Output written to {os.path.abspath(save_root)}')

result = face_reconstruction(
    'https://modelscope.oss-cn-beijing.aliyuncs.com/test/images/face_reconstruction.jpg'
)
save_results(result, './face_reconstruction_results')
print('✅ HRN 추론 성공!')

In [ ]:
# ============================================================
# Cell 5: 결과 시각화
# ============================================================
from matplotlib import pyplot as plt

img = cv2.imread('./face_reconstruction_results/vis_image.jpg', -1)
plt.figure(figsize=(12, 6))
plt.imshow(img[..., ::-1])
plt.axis('off')
plt.title('HRN 3D Face Reconstruction Result')
plt.show()
print('✅ 끝! HRN이 정상 동작합니다.')
print(f'   다음 런타임 재시작 시 Drive 캐시로 ~2분 안에 복구됩니다.')